# <center> <img src="./img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Proyecto Final: Website Activity** </center>
---

<center>

**Equipo McQueen**: Marco Albanese, Vicente Siloe

**Carrera**: Ingeniería en Sistemas Computacionales

**Fecha**: 13 de mayo del 2025

**Profesor**: Pablo Camarillo Ramirez

</center>

---
### <center> **Primavera 2025** </center>
---

### **Introducción y definición del problema**

El procesamiento de datos en aplicaciones es esencial hoy en día para una multitud de usos y propósitos que proveen beneficios tanto a usuarios como empresas. En este proyecto, creado al culminar el curso de "Procesamiento de datos masivos", se pretende crear una aplicación que integre los conocimientos adquiridos en dicho curso. A través de una *pipeline* de *Big Data*, se busca producir, consumir, analizar y procesar datos de la actividad (tráfico) observada en una aplicación web.

Los datos involucrados representan:
- Interacciones de usuario
- Páginas visitadas
- Clicks
- Dispositivos utilizados
- Etc.

Asimismo, se entrena un modelo de aprendizaje automático (*machine learning*) en base a los datos recolectados para predecir comportamientos de usuario, utilizando regresión logística. Los resultados de este modelo se presentan al usuario final a través de un *dashboard* de visualización de datos.

Para la realización de este proyecto, se utilizaron tecnologías como **Jupyter Notebooks**, **Python**, **PySpark**, **Apache Kafka** y **Power BI**. En secciones posteriores, se describe con mayor detalle la arquitectura, implementación y funcionamiento del sistema.

### **Arquitectura del sistema**

La arquitectura del sistema es la siguiente:

1. Productores:
2. Consumidor:
3. Spark
4. Modelo de aprendizaje máquina (Machine Learning)
5. Visualización de datos:

A continuación se presenta un diagrama visual de dicha arquitectura:


# <img src="./img/system_architecture.png" alt="System Architecture">

### **Justificación de las 5Vs**

- Volumen:
- Velocidad:
- Variedad:
- Veracidad:
- Valor:

### **Detalles de implementación**

Placeholder

### **Resultados y evaluación**

Placeholder

### **Conclusión**

Placeholder

---

### **Código: Spark + Kafka**
---

Previo a la ejecución del siguiente código, los contenedores de Docker para el cluster de Spark y el cluster de Kafka deben estar activos.

Para ello, se puede utilizar el comando:

```
docker compose up
```

En los directorios `spark_cluster` y `kafka_cluster`. Notesé que se requerirán distintos IDs de contenedores en los bloques de código pertinentes.

#### Encontrar instalación de Spark

In [ ]:
import findspark
findspark.init()

#### Creación de la conexión con el cluster de Spark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Final-Project-Equipo-McQueen") \
    .master("spark://f8e470126b03:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()

sc = spark.sparkContext

#### Creación de tópicos de Kafka

Los siguientes comandos deben ser ejecutados en la terminal para que los productores puedan enviar mensajes a los tópicos:

```
docker exec -it <kafka container id> /opt/kafka/bin/kafka-topics.sh --create --zookeeper zookeeper:2181 --replication-factor 1 --partitions 1 --topic producer-a

docker exec -it <kafka container id> /opt/kafka/bin/kafka-topics.sh --create --zookeeper zookeeper:2181 --replication-factor 1 --partitions 1 --topic producer-b
```

#### Ejecución de productores

Para inicializar a los productores, ejecute los respectivos scripts de Python bajo el directorio `spark_cluster/notebooks/lib/equipo_mcqueen/` de la siguiente manera:

`python producer-a.py`

`python producer-b.py`

#### Creación del Kafka stream

In [ ]:
kafka_lines = spark \
                .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "9bc654f7b2fc:9093") \
                .option("subscribe", "producer-a", "producer-b") \
                .load()

kafka_lines.printSchema()

#### Transformar datos en cadenas (strings)

In [ ]:
kafka_df = kafka_lines.withColumn("value_str", kafka_lines.value.cast("string"))

#### Creación del query (sink)

In [ ]:
query = kafka_df \
                .writeStream \
                .trigger(processingTime="3 seconds") \
                .outputMode("append") \
                .format("parquet") \
                .option("path", "/home/jovyan/notebooks/data/") \
                .option("checkpointLocation", "/home/jovyan/checkpoint") \
                .option("truncate", "false") \
                .start()

query.awaitTermination(200)

#### Terminación de query y context

In [ ]:
query.stop()
sc.stop()

### **Código: Machine Learning**
---

#### Creación de la conexión con el cluster de Spark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Equipo-McQueen-Logistic-Regression") \
    .master("spark://078b2e28e517:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()

sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

#### Preparación de datos

#### Creación de vector

In [ ]:
from pyspark.ml.feature import VectorAssembler

#### División de datos para entrenamiento y pruebas

#### Creación y entrenamiento del modelo (regresión logística)

In [ ]:
from pyspark.ml.classification import LogisticRegression

#### Evaluación y predicciones del modelo

### **Visualización de datos: Power BI**
---

#### Conversión a CSV

#### Dashboard de ejemplo

Placeholder